In [2]:
# Create directory tree

from pathlib import Path

root_dir = Path.cwd()

dirs_to_create = [
  root_dir / "databases" / "MarcosLaffitte",
  root_dir / "databases" / "BrendanMcKay",
  root_dir / "cache"]

# Create directories
for dir_path in dirs_to_create:
  dir_path.mkdir(parents=True, exist_ok=True)
  print(f"Created directory: {dir_path}")

Created directory: c:\Users\Tonatiuh\OneDrive\Documentos\GitHub\FerGroup\databases\MarcosLaffitte
Created directory: c:\Users\Tonatiuh\OneDrive\Documentos\GitHub\FerGroup\databases\BrendanMcKay
Created directory: c:\Users\Tonatiuh\OneDrive\Documentos\GitHub\FerGroup\cache


In [3]:
# Download Marcos Laffitte's databases

import requests

url = "https://github.com/MarcosLaffitte/Amoebas/raw/main/6_Examples/c_Bigger_Analyses/"

marcos_file_names = [
  "Amoeba_Graphs_1_to_10_vertices_raw.pkl",
  "Amoeba_Trees_1_to_22_vertices_raw.pkl"]
  
for file_name in marcos_file_names:
  response = requests.get(url + file_name)
  if response.status_code == 200:
    with open("databases/MarcosLaffitte/" + file_name, "wb") as file:
      file.write(response.content)
  else:
    print("Failed to download file:", response.status_code)

In [2]:
# Download Brendan McKay's databases (currently only homeomorphically irreducible trees)
# More graphs can be found here https://users.cecs.anu.edu.au/~bdm/data/graphs.html
# But the amoebas are likely already contained above.
# More trees found here https://houseofgraphs.org/meta-directory/trees. Same situation.

import requests

url = "https://users.cecs.anu.edu.au/~bdm/data/"

brendan_file_names = [
  "graph2c.g6",
  "graph3c.g6",
  "graph4c.g6",
  "graph5c.g6",
  "graph6c.g6",
  "graph7c.g6",
  "graph8c.g6",
  "graph9c.g6",
  "hitree23.txt",
  "hitree24.txt",
  "hitree25.txt"]
  
for file_name in brendan_file_names:
  response = requests.get(url + file_name)
  if response.status_code == 200:
    with open("databases/BrendanMcKay/" + file_name, "wb") as file:
      file.write(response.content)
  else:
    print("Failed to download file:", response.status_code)

In [24]:
# [WORK IN PROGRESS] Insert Brendan McKay's data

import networkx as nx

# Parse text into edges
def parse_edges(text):
  edge_list = []
  parts = text.strip().split()
  for i in range(0, len(parts), 2):
    edge_list.append((int(parts[i]), int(parts[i + 1])))
  return edge_list

# Convert graph to g6 WARNING: all entries have prefix >>graph6<<
def convert_to_g6(graph):
  return nx.to_graph6_bytes(graph).decode('utf-8').strip()

conn = sqlite3.connect('databases/all_graphs.db')
cursor = conn.cursor()

for file_name in brendan_file_names:
  with open("databases/BrendanMcKay/"+file_name, 'r') as file:
    for line in file:
      edges = parse_edges(line)
      num_edges = len(edges)
      G = nx.Graph()
      G.add_edges_from(edges)
      graph_g6 = convert_to_g6(G)
      num_nodes = len(G.nodes())
      is_connected = nx.is_connected(G)
      
      db_write(conn, graph_g6, num_nodes, num_edges, is_connected, True)

conn.close()

In [27]:
# Read database

conn = sqlite3.connect('databases/graphs.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM graphs WHERE graph_g6 = ?', ('>>graph6<<VhCGOOGA?OA?G?O?O???@??C??G??O??O??O??GC????',))
row = cursor.fetchone()
print(row)

conn.close()

(29690, '>>graph6<<VhCGOOGA?OA?G?O?O???@??C??G??O??O??O??GC????', 23, 22, 1, 1, None, None, None)
